In [ ]:
import numpy as np
from pandas import DataFrame
import keras
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
# %matplotlib notebook

# Récupération et mise en forme des données

In [ ]:
X = np.load('X.npy')
Y = np.load('Y.npy')

In [ ]:
X.shape

In [ ]:
X_flat = np.array(X).reshape((2062, 64*64))
X_flat.shape

In [ ]:
Ydf = DataFrame(data=Y, columns=[9-x for x in range(10)])
Ydf.head()

In [ ]:
Ydf = Ydf[[x for x in range(10)]]
Ydf.head()

In [ ]:
Ydf['class']=(Ydf[range(10)].idxmax(axis=1)).astype('int')
Ydf[['class']]

In [ ]:
Y_flat = Ydf['class'].to_numpy()
Y_flat.shape

In [ ]:
Y_flat = Y_flat.reshape(-1,1)
Y_flat.shape

In [ ]:
from numpy import random as np_random

SEED = 42
np_random.seed(SEED)

In [ ]:
from sklearn.model_selection import train_test_split

X_train_CNN, X_val_CNN, y_train_CNN, y_val_CNN = train_test_split(X, Y_flat, test_size=0.2)
X_train_CNN.shape

In [ ]:
X_val_CNN.shape

In [ ]:
y_train_CNN.shape

In [ ]:
X_train_CNN_reshape = X_train_CNN.reshape(1649,64,64,1)
X_train_CNN_reshape.shape

In [ ]:
X_val_CNN_reshape = X_val_CNN.reshape(413,64,64,1)
X_val_CNN_reshape.shape

# Model CNN

In [ ]:
# LEARNING_RATE = 0.001
# BATCH_SIZE = X_train_CNN.shape[1]
# EPOCHS = 50

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout #create model

model_CNN = Sequential() #add model layers

model_CNN.add(Conv2D(8, kernel_size=3, 
                     activation='relu', 
                     input_shape=(64,64,1), 
                     kernel_initializer='glorot_normal'))
model_CNN.add(MaxPooling2D(pool_size=(2, 2)))
model_CNN.add(Conv2D(16, kernel_size=3, 
                     activation='relu', 
                     kernel_initializer='glorot_normal'))
model_CNN.add(MaxPooling2D(pool_size=(2, 2)))
model_CNN.add(Conv2D(32, kernel_size=3, 
                     activation='relu', 
                     kernel_initializer='glorot_normal'))
model_CNN.add(MaxPooling2D(pool_size=(2, 2)))
model_CNN.add(Conv2D(64, kernel_size=3, 
                     activation='relu', 
                     kernel_initializer='glorot_normal'))
model_CNN.add(MaxPooling2D(pool_size=(2, 2)))
model_CNN.add(Flatten())
model_CNN.add(Dense(32, activation='relu'))
model_CNN.add(Dropout(0.2))
# model_CNN.add(Dense(16, activation='relu'))
# model_CNN.add(Dropout(0.2))
model_CNN.add(Dense(10, activation='softmax'))

In [ ]:
model_CNN.summary()

In [ ]:
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.optimizers import RMSprop

model_CNN.compile(loss='sparse_categorical_crossentropy', 
                  optimizer=Adam(lr=LEARNING_RATE),
                  metrics = ['accuracy']) #optimizer=SGD(lr=LEARNING_RATE), optimizer=Adam(lr=LEARNING_RATE)

In [ ]:
history_CNN = model_CNN.fit(X_train_CNN_reshape,
                            y_train_CNN,
                            validation_data=(X_val_CNN_reshape, y_val_CNN),
                            epochs=EPOCHS,
                            batch_size=BATCH_SIZE,
                            verbose=1)

In [ ]:
pd.DataFrame(history_CNN.history).plot()
plt.grid(True)
plt.title('Model performance throughout training CNN')
plt.ylabel('metrics')
plt.xlabel('epoch')
plt.legend(['val_loss', 'val_acc', 'train_loss', 'train_acc'], loc='upper right')
plt.show()

# Evaluate CNN

In [ ]:
CNN_pred_val = model_CNN.predict(X_val_CNN_reshape)
CNN_pred_val

In [ ]:
df_CNN_pred_val = DataFrame(CNN_pred_val, columns=[x for x in range(10)])
df_CNN_pred_val.head()

In [ ]:
df_CNN_pred_val['class_CNN_pred']=(df_CNN_pred_val[range(10)].idxmax(axis=1)).astype('int')
df_CNN_pred_val.head()

In [ ]:
y_CNN_pred_val = df_CNN_pred_val['class_CNN_pred'].to_numpy()
y_CNN_pred_val = y_CNN_pred_val.reshape(-1,1)

In [ ]:
y_CNN_pred_val.shape

In [ ]:
y_val_CNN.shape

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_val_CNN, y_CNN_pred_val)

## Méthode pour exporter ce CNN dans un autre notebook

In [ ]:
def cnn_model(inputs, output, OPTIMIZER, LEARNING_RATE, EPOCHS, BATCH_SIZE, nb_dense, ACTIVATION, INITIALIZER):
    X_train, X_val, y_train, y_val = train_test_split(inputs, output, test_size=0.2)
    
    model = Sequential() #add model layers

    model.add(Conv2D(8, kernel_size=3, 
                     activation=ACTIVATION, 
                     input_shape=(inputs.shape[1],inputs.shape[2],inputs.shape[3]), 
                     kernel_initializer=INITIALIZER))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(16, kernel_size=3, 
                     activation=ACTIVATION, 
                     kernel_initializer=INITIALIZER))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, kernel_size=3, 
                     activation=ACTIVATION, 
                     kernel_initializer=INITIALIZER))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=3, 
                     activation=ACTIVATION, 
                     kernel_initializer=INITIALIZER))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(32, activation=ACTIVATION))
    model.add(Dropout(0.2))
# model.add(Dense(16, activation='relu'))
# model.add(Dropout(0.2))
    model.add(Dense(nb_dense, activation='softmax'))
    
    model.compile(loss='sparse_categorical_crossentropy', 
                    optimizer=OPTIMIZER(lr=LEARNING_RATE), 
                    metrics = ['accuracy'])
    
    history = model.fit(X_train,
                        y_train,
                        validation_data=(X_val, y_val),
                        epochs=EPOCHS,
                        batch_size=BATCH_SIZE,
                        verbose=2)
    
    pd.DataFrame(history.history).plot()
    plt.grid(True)
    plt.title('Model performance throughout training CNN')
    plt.ylabel('metrics')
    plt.xlabel('epoch')
    plt.legend(['val_loss', 'val_acc', 'train_loss', 'train_acc'], loc='upper right')
    plt.show()
    return model